## Imports

In [123]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import os
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

## DB connection

In [9]:
load_dotenv(override=True)
print(os.getenv("DB_USER"))
print(os.getenv("DB_PASSWORD"))
print(os.getenv("DB_HOST"))
print(os.getenv("DB_PORT"))

readonly
kN2E2NjhjNTU0NmFkIg==
localhost
5434


In [10]:
# Define connection parameters from environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")

# Create the connection string
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"

# Create an engine
engine = create_engine(connection_string)

# Test connection
with engine.connect() as conn:
    result = conn.execute("SELECT version();")
    for row in result:
        print(row)

('PostgreSQL 17.5 on x86_64-pc-linux-musl, compiled by gcc (Alpine 14.2.0) 14.2.0, 64-bit',)


## Queries

### Selecting 1-bedroom apartments with 2+ images in the ad in the Kolonos district

In [11]:
query = """ WITH polygon AS (
  SELECT ST_SetSRID(ST_MakePolygon(ST_GeomFromText('LINESTRING(
      23.695234901846018 37.996147960821915,
      23.715207211595587 38.00777143269395,
      23.714032369845615 37.98544862411932,
      23.72147303426212 37.996970922085865,
      23.695234901846018 37.996147960821915
  )')), 4326) AS poly
)
SELECT 
                      p.id,
                      p.url,
                      p.price,
                      p.square,
                      p.floor,
                      p.bedrooms,
                      p.city,
                      p.country,
                      p.district,
                      p.is_sold,
                      p.publication_date,
                      p.modification_date,
                      p.created_at,
                      p.year_built,
                      p.location_name,
                      p.bathrooms,
                      p.realtor_name,
                      p.realtor_url,
                      p.property_type,
                      p.last_review_date,
                      p.photos_folder_url,
                      p.latitude,
                      p.longitude,
                      r.group_property_id
FROM properties p
LEFT JOIN properties__group_properties_relations r ON r.property_id = p.id,
     polygon
WHERE 
  p.num_photos >= 2
  AND p.property_type = 'apartment'
  AND p.bedrooms = 1
  AND geo_point && poly 
  AND ST_Contains(poly, geo_point::geometry)
ORDER BY p.created_at DESC;
"""

In [12]:
df = pd.read_sql(query, engine)

## Analysis

### price per sqm calculation and floor normalization

In [103]:
df['price_per_sqm'] = df['price'] / df['square']

def normalize_floor(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip().lower()
    if 'ground floor' in val:
        return 'GF'
    if 'mezzanine' in val:
        return 'MEZ'
    
    match = re.match(r"(\d+)(st|nd|rd|th)?", val)
    if match:
        return int(match.group(1))
    
    return val  # Return original if no match

In [104]:
df['floor'] = df['floor'].apply(normalize_floor)

In [105]:
csv = df.to_csv('kolonos.csv', index=False)

### Filtering multi-listings duplciates

In [106]:
df['group_property_id'].replace('', pd.NA, inplace=True)

In [107]:
# Separate rows with and without group_property_id
with_group = df[df['group_property_id'].notna()]
without_group = df[df['group_property_id'].isna()]

In [108]:
# Keep only one row per group_property_id
with_group_unique = with_group.drop_duplicates(subset='group_property_id')
# Combine back together
result_df = pd.concat([with_group_unique, without_group], ignore_index=True)


In [109]:
result_df

,id,url,price,square,floor,bedrooms,city,country,district,is_sold,...,bathrooms,realtor_name,realtor_url,property_type,last_review_date,photos_folder_url,latitude,longitude,group_property_id,price_per_sqm
0,b5c6e3e2-040b-4d17-8665-4a4f2320139f,https://www.xe.gr/en/property/d/property-to-re...,580.0,71.0,1,1,Athens,Greece,Larissis station,False,...,1,D-Home Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-11 19:08:41.934537+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.995130,23.718899,97708b58-5e2a-4904-9058-bf3230ef38fd,8.169014
1,e0b66239-f9c8-44d6-a846-c01e585c0f58,https://www.xe.gr/en/property/d/property-to-re...,470.0,70.0,1,1,Athens,Greece,Kolonos,False,...,1,Now Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-06 20:04:45.882531+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.988186,23.715191,9628eb25-dc74-4bbe-ae84-2be784202d35,6.714286
2,14bed563-b3db-4570-9cbe-c4deac38b7d1,https://www.xe.gr/en/property/d/property-to-re...,470.0,42.0,1,1,Athens,Greece,Larissis station,False,...,1,,,apartment,2025-06-06 20:03:50.900074+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.994675,23.718777,b16180f6-6a97-4654-a4ae-0c5e00f91235,11.190476
3,d4339626-04dd-4306-af3d-7bd353fadff6,https://www.xe.gr/en/property/d/property-for-s...,175000.0,55.0,4,1,Athens,Greece,Akadimia Platonos,False,...,1,Lionbridge Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-06 19:57:08.962884+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.996908,23.709624,bec4f936-6461-4cf8-b69a-5528e32c6969,3181.818182
4,a169dbb0-032e-46d6-bc94-8899e5bf7ba2,https://www.xe.gr/en/property/d/property-to-re...,650.0,98.0,2,1,Athens,Greece,Kolonos,False,...,1,Now Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-02 01:02:08.890413+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.991107,23.717601,2ae5a8e4-6f3b-4cc2-bf28-f7ed96488e1b,6.632653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,d7a9c840-c683-42c6-8d30-448302e92aea,https://www.xe.gr/en/property/d/property-for-s...,98500.0,50.0,3,1,Athens,Greece,Kolokinthou,True,...,1,Planet Homes,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2023-12-23 02:26:29.051677+00:00,None,37.998815,23.710953,None,1970.000000
599,e5440f0b-09fa-4ec5-8e2f-0c25764d296a,https://www.xe.gr/en/property/d/property-for-s...,85000.0,52.0,3,1,Athens,Greece,Kolonos,True,...,1,Public Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2024-01-08 13:19:28.745561+00:00,None,37.993563,23.715621,None,1634.615385
600,c8d0bde1-5b04-48d8-8639-8222234aca9f,https://www.xe.gr/en/property/d/property-for-s...,103000.0,48.0,3,1,Athens,Greece,Sepolia,True,...,1,Planet Homes,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2023-12-17 18:56:35.750578+00:00,None,38.005694,23.715087,None,2145.833333
601,fe4daaa7-235f-4b3d-8685-3429f86051d0,https://www.xe.gr/en/property/d/property-for-s...,98500.0,50.0,3,1,Athens,Greece,Kolokinthou,True,...,1,Planet Homes,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2023-12-25 13:29:41.965800+00:00,None,37.998815,23.710953,None,1970.000000


### On-sale and LTR dfs

In [110]:
comps_ldr_df = df[df['price_per_sqm'] < 100]
comps_sale_df = df[df['price_per_sqm'] > 2000]

In [129]:
comps_ldr_df

,id,url,price,square,floor,bedrooms,city,country,district,is_sold,...,realtor_name,realtor_url,property_type,last_review_date,photos_folder_url,latitude,longitude,group_property_id,price_per_sqm,sqm_category
0,b5c6e3e2-040b-4d17-8665-4a4f2320139f,https://www.xe.gr/en/property/d/property-to-re...,580.0,71.0,1,1,Athens,Greece,Larissis station,False,...,D-Home Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-11 19:08:41.934537+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.995130,23.718899,97708b58-5e2a-4904-9058-bf3230ef38fd,8.169014,60+
1,e0b66239-f9c8-44d6-a846-c01e585c0f58,https://www.xe.gr/en/property/d/property-to-re...,470.0,70.0,1,1,Athens,Greece,Kolonos,False,...,Now Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-06 20:04:45.882531+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.988186,23.715191,9628eb25-dc74-4bbe-ae84-2be784202d35,6.714286,60+
2,14bed563-b3db-4570-9cbe-c4deac38b7d1,https://www.xe.gr/en/property/d/property-to-re...,470.0,42.0,1,1,Athens,Greece,Larissis station,False,...,,,apartment,2025-06-06 20:03:50.900074+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.994675,23.718777,b16180f6-6a97-4654-a4ae-0c5e00f91235,11.190476,40-60
9,a169dbb0-032e-46d6-bc94-8899e5bf7ba2,https://www.xe.gr/en/property/d/property-to-re...,650.0,98.0,2,1,Athens,Greece,Kolonos,False,...,Now Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-02 01:02:08.890413+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.991107,23.717601,2ae5a8e4-6f3b-4cc2-bf28-f7ed96488e1b,6.632653,60+
10,5cd51315-d0fe-4c7c-979c-b98b8e0ad315,https://www.xe.gr/en/property/d/property-to-re...,500.0,35.0,4,1,Athens,Greece,Kolonos,False,...,,,apartment,2025-06-02 00:59:37.983190+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.992355,23.716753,None,14.285714,20-40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,55d0f7ab-17be-4a42-a2c9-c600887c9b24,https://www.xe.gr/en/property/d/property-to-re...,500.0,55.0,3,1,Athens,Greece,Kolonos,True,...,VIProperty,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2024-03-15 07:13:44.516779+00:00,None,37.992506,23.714805,None,9.090909,40-60
682,ca6f2483-9296-400d-82a2-f02cd41a097c,https://www.xe.gr/en/property/d/property-to-re...,500.0,45.0,1,1,Athens,Greece,Kolokinthou,False,...,Palmos Properties,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2024-03-15 05:32:18.633117+00:00,None,38.000188,23.709668,None,11.111111,40-60
683,996ae8b2-86e6-4e44-9384-92d57d9a0d95,https://www.xe.gr/en/property/d/property-to-re...,550.0,52.0,1,1,Athens,Greece,Larissis station,True,...,Zirogiannis Real Estate Specialists,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2024-06-10 17:33:55.936799+00:00,None,37.995497,23.719988,None,10.576923,40-60
684,c91003c1-c55c-4c8b-8cd9-558e2a74568b,https://www.xe.gr/en/property/d/property-to-re...,450.0,42.0,2,1,Athens,Greece,Kolokinthou,False,...,Aspis Real Estate Περιστέρι,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2024-03-15 04:30:21.192630+00:00,None,37.996941,23.711657,None,10.714286,40-60


In [131]:
comps_sale_df = df[df['price_per_sqm'] > 2000]
comps_sale_df

,id,url,price,square,floor,bedrooms,city,country,district,is_sold,...,bathrooms,realtor_name,realtor_url,property_type,last_review_date,photos_folder_url,latitude,longitude,group_property_id,price_per_sqm
5,d4339626-04dd-4306-af3d-7bd353fadff6,https://www.xe.gr/en/property/d/property-for-s...,175000.0,55.0,4,1,Athens,Greece,Akadimia Platonos,False,...,1,Lionbridge Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-06 19:57:08.962884+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.996908,23.709624,bec4f936-6461-4cf8-b69a-5528e32c6969,3181.818182
7,1241eefc-a2de-4804-a009-ddb93da1d020,https://www.xe.gr/en/property/d/property-for-s...,78000.0,30.0,mez,1,Athens,Greece,Kolonos,False,...,1,One Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-06 19:56:18.469988+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.990711,23.716479,None,2600.000000
17,69a599e1-b2de-4882-8ee2-b4143e9b3dc4,https://www.xe.gr/en/property/d/property-for-s...,102000.0,48.0,1,1,Athens,Greece,Kolokinthou,False,...,1,Family Brokers,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-06-02 00:51:43.150299+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,38.000922,23.711154,None,2125.000000
32,f7cd5626-e340-4fd4-a5c2-46301dbb4895,https://www.xe.gr/en/property/d/property-for-s...,90000.0,40.0,2,1,Athens,Greece,Kolokinthou,False,...,1,Wecan Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-05-22 04:47:08.633456+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,37.999929,23.710590,a61e2f3a-426a-42ef-b18a-e7b09146bfc6,2250.000000
33,1dcaa712-e16c-4103-be36-a5631348b338,https://www.xe.gr/en/property/d/property-for-s...,90000.0,40.0,2,1,Athens,Greece,Kolokinthou,False,...,1,Mg Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2025-05-22 04:46:41.683070+00:00,https://filer-seaweedfs.kube.cluster1.renty.li...,38.000259,23.711958,a61e2f3a-426a-42ef-b18a-e7b09146bfc6,2250.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,f442c7f1-d164-457f-9b4e-31cbb59b96d7,https://www.xe.gr/en/property/d/property-for-s...,177000.0,59.0,1,1,Athens,Greece,Akadimia Platonos,True,...,1,Utopia Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2024-02-11 12:00:53.935308+00:00,None,37.987393,23.715059,None,3000.000000
729,6735aeed-3425-475c-93b6-17d9ca85921f,https://www.xe.gr/en/property/d/property-for-s...,250000.0,67.0,1,1,Athens,Greece,Larissis station,True,...,1,Morcos Real Estate,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2024-02-11 08:10:18.593477+00:00,None,37.994812,23.718849,None,3731.343284
730,cfb35948-6c0b-44ae-bfbd-b27d344ba499,https://www.xe.gr/en/property/d/property-for-s...,205000.0,50.0,4,1,Athens,Greece,Larissis station,True,...,1,Downtown Properties,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2024-08-02 13:38:41.103993+00:00,https://console-storage.renty.life/browser/pho...,37.994802,23.718807,None,4100.000000
731,45ffbe3f-9126-401e-a5ce-95c60670b904,https://www.xe.gr/en/property/d/property-for-s...,103000.0,48.0,3,1,Athens,Greece,Sepolia,True,...,1,Planet Homes,https://www.xe.gr/en/property/s/real-estate-ag...,apartment,2023-12-25 23:21:12.806959+00:00,None,38.005694,23.715087,None,2145.833333


### Price analysis

In [111]:
def categorize_sqm(sqm):
    try:
        s = float(sqm)
        if 20 <= s < 40:
            return '20-40'
        elif 40 <= s < 60:
            return '40-60'
        elif s >= 60:
            return '60+'
        else:
            return 'Below 20'
    except:
        return 'Unknown'

In [115]:
comps_ldr_df['sqm_category'] = comps_ldr_df['square'].apply(categorize_sqm)
comps_sale_df['sqm_category'] = comps_sale_df['square'].apply(categorize_sqm)

/var/folders/1v/v0zl284d0158znzld0fnfh140000gn/T/ipykernel_22784/2739212004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comps_ldr_df['sqm_category'] = comps_ldr_df['square'].apply(categorize_sqm)
/var/folders/1v/v0zl284d0158znzld0fnfh140000gn/T/ipykernel_22784/2739212004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comps_sale_df['sqm_category'] = comps_sale_df['square'].apply(categorize_sqm)


In [119]:
percentiles = [0.5, 0.75, 0.9]
grouped_ltr = comps_ldr_df.groupby(['sqm_category', 'floor'])
grouped_on_sale = comps_sale_df.groupby(['sqm_category', 'floor'])

In [120]:
# Aggregate
summary_ltr = grouped_ltr['price_per_sqm'].agg(
    count='count',
   p50=lambda x: round(x.quantile(0.5), 0),
    p75=lambda x: round(x.quantile(0.75), 0),
    p90=lambda x: round(x.quantile(0.9), 0)
).reset_index()

# Aggregate
summary_sale = grouped_on_sale['price_per_sqm'].agg(
    count='count',
   p50=lambda x: round(x.quantile(0.5), 0),
    p75=lambda x: round(x.quantile(0.75), 0),
    p90=lambda x: round(x.quantile(0.9), 0)
).reset_index()

In [121]:
summary_ltr

,sqm_category,floor,count,p50,p75,p90
0,20-40,1,7,12.0,13.0,13.0
1,20-40,2,8,11.0,12.0,14.0
2,20-40,3,10,13.0,16.0,17.0
3,20-40,4,6,12.0,13.0,14.0
4,20-40,5,5,17.0,30.0,32.0
5,20-40,gf,12,14.0,15.0,16.0
6,20-40,mez,3,12.0,12.0,13.0
7,40-60,1,19,10.0,11.0,12.0
8,40-60,2,12,11.0,12.0,12.0
9,40-60,3,11,10.0,11.0,11.0


In [122]:
summary_sale

,sqm_category,floor,count,p50,p75,p90
0,20-40,1,7,2364.0,2450.0,2780.0
1,20-40,2,11,2200.0,2461.0,2600.0
2,20-40,3,2,2390.0,2445.0,2478.0
3,20-40,4,5,2067.0,2067.0,2750.0
4,20-40,5,4,2692.0,2724.0,2782.0
5,20-40,6,3,2632.0,2632.0,2632.0
6,20-40,gf,8,2190.0,3061.0,5667.0
7,20-40,mez,8,2600.0,2600.0,2600.0
8,40-60,1,31,2545.0,2927.0,3700.0
9,40-60,2,29,2375.0,2500.0,3076.0


### Heat maps

In [127]:
# Pivot the summary table to have a multi-indexed column (sqm_category, metric)
styled_sale_summary = summary_sale.pivot(index='floor', columns='sqm_category')
styed_ltr_summary = summary_ltr.pivot(index='floor', columns='sqm_category')

# Reorder columns so 'count' appears first for each sqm_category (if needed)
styled_sale_summary = styled_sale_summary.reorder_levels([1, 0], axis=1).sort_index(axis=1)
styed_ltr_summary = styed_ltr_summary.reorder_levels([1, 0], axis=1).sort_index(axis=1)


# Create a Styler object
def heatmap_on_percentiles(styler):
    percentiles = ['p50', 'p75', 'p90']
    # Apply background gradient only to percentile rows
    for col in percentiles:
        cols_to_color = [c for c in styler.columns if c[1] == col]
        styler = styler.background_gradient(
            subset=cols_to_color,
            cmap="YlOrRd"
        )
    return styler

styled_sale = styled_sale_summary.style.pipe(heatmap_on_percentiles).format(precision=0)
styled_ltr = styed_ltr_summary.style.pipe(heatmap_on_percentiles).format(precision=0)

# Display in Jupyter or Streamlit (or export)
styled_sale


In [128]:
styled_ltr